In [1]:
#| default_exp interface_trimesh

In [2]:
#| export

from blender_tissue_cartography import mesh as tcmesh

import numpy as np
import trimesh

In [3]:
from importlib import reload

In [4]:
np.set_printoptions(suppress=True)

## Interfacing with `trimesh`

In this notebook, we define functions to convert our `ObjMesh` class to and from `trimesh`.

Note: `trimesh` represents triangular meshes only, and its way for representing UV information is not ideal. It is not recommended to edit mesh topology in `trimesh` if the UV mapping has already been defined.

In [7]:
mesh_fname_data = "registration_example/Drosophila_CAAX-mCherry_mesh_remeshed.obj"
mesh_fname_ref = "registration_example/Drosophila_reference.obj"

In [8]:
mesh_data = tcmesh.ObjMesh.read_obj(mesh_fname_data)
mesh_ref = tcmesh.ObjMesh.read_obj(mesh_fname_ref)

/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/io.py:202: RuntimeWarning: Warning: multiple meshes in .obj file
  warnings.warn(f"Warning: multiple meshes in .obj file", RuntimeWarning)


In [9]:
#| export

def convert_to_trimesh(mesh: tcmesh.ObjMesh, add_texture_info=None) -> trimesh.Trimesh:
    """
    Convert tcmesh.ObjMesh to trimesh.Trimesh
    
    See https://trimesh.org/trimesh.base.html
    Note: normal information is recalculated. Discards any non-triangle faces.
    
    Texture is saved as a vertex attribute via v_tex_coords_matrix. Note that this discards
    information since a vertex can have multiple texture coordinates!
    For this reason, we also add the texture coordinates as a (n_faces, 3, s)-array attribute
    `face_tex`. Note: this will _not_ be updated if you remesh etc.
    
    Parameters
    ----------
    mesh : tcmesh.ObjMesh
    add_texture_info : None or bool
        Whether to add texture info to the trimesh.Trimesh. If None, texture is added if
        available for at least one vertex.
    Returns
    -------
    trimesh.Trimesh

    """
    if not mesh.is_triangular:
        warnings.warn(f"Warning: mesh not triangular. discarding non-triangular faces")
    add_texture_info = ((not mesh.only_vertices and len(mesh.texture_vertices) > 0)
                        if add_texture_info is None else add_texture_info)
    if not add_texture_info:
        return trimesh.Trimesh(mesh.vertices, mesh.tris)
    texture = trimesh.visual.texture.TextureVisuals(uv=mesh.vertex_textures)
    converted = trimesh.Trimesh(mesh.vertices, mesh.tris, visual=texture)
    converted.face_tex = tcmesh.index_else_nan(mesh.texture_vertices, mesh.texture_tris)
    return converted

In [10]:
trimesh_data = convert_to_trimesh(mesh_data)

In [11]:
%%time
trimesh_ref = convert_to_trimesh(mesh_ref)

CPU times: user 339 ms, sys: 1.33 ms, total: 340 ms
Wall time: 340 ms


In [12]:
np.allclose(mesh_ref.vertices, trimesh_ref.vertices)

True

In [14]:
trimesh_normals = trimesh_ref.vertex_normals
trimesh_normals = (trimesh_normals.T / np.linalg.norm(trimesh_normals, axis=-1)).T

np.einsum('vi,vi->v', mesh_ref.normals, trimesh_normals)

array([0.99999999, 0.9999961 , 0.99999858, ..., 0.99999886, 0.9999949 ,
       1.        ])

In [15]:
np.allclose(mesh_ref.vertex_textures, trimesh_ref.visual.uv)

True

In [16]:
%%timeit
convert_to_trimesh(mesh_ref)

251 ms ± 4.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
#| export

def convert_from_trimesh(mesh: trimesh.Trimesh, reconstruct_texture_from_faces=True,
                           texture_vertex_decimals=10) -> tcmesh.ObjMesh:
    """
    Convert trimesh mesh to ObjMesh.
    
    Texture vertices can be reconstructed from face attribute face_tex or from
    vertex attribute vertex_tex_coord_matrix. Reconstruction from face texture can accomodate
    multiple texture coordinates per vertex (e.g. for UV maps with seams).
    
    Texture vertices are rounded to texture_vertex_decimals decimals
    """
    vertices = mesh.vertices
    normals = mesh.vertex_normals
    normals = (normals.T / np.linalg.norm(normals, axis=-1)).T
    if not hasattr(mesh.visual, 'uv'):
        faces = [[2*[v,] for v in f] for f in mesh.faces]
        return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals)
    if hasattr(mesh.visual, 'uv') and not reconstruct_texture_from_faces:
        faces = [[2*[v,] for v in f] for f in mesh.faces]
        return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=mesh.visual.uv)
    # reconstruct texture vertices - big pain.
    texture_vertices = np.vstack([mesh.face_tex[:,i,:] for i in [0,1,2]])
    texture_vertices = np.round(texture_vertices, decimals=texture_vertex_decimals)
    texture_vertices_unique, inverse_index = np.unique(texture_vertices, axis=0, return_inverse=True)

    n_faces = mesh.faces.shape[0]
    faces = [[[v, inverse_index[ifc+iv*n_faces]] for iv, v in enumerate(fc)]
              for ifc, fc in enumerate(mesh.faces)]

    return tcmesh.ObjMesh(vertices=vertices, faces=faces, normals=normals, texture_vertices=texture_vertices_unique)

In [18]:
trimesh_ref = convert_to_trimesh(mesh_ref)

In [19]:
%%time
convert_from_trimesh(trimesh_ref)

CPU times: user 460 ms, sys: 28.2 ms, total: 488 ms
Wall time: 485 ms


In [20]:
mesh_seams = tcmesh.ObjMesh.read_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv.obj")

/home/nikolas/Documents/UCSB/streichan/numerics/code/python code/jupyter notebooks/blender-tissue-cartography/blender_tissue_cartography/io.py:202: RuntimeWarning: Warning: multiple meshes in .obj file
  warnings.warn(f"Warning: multiple meshes in .obj file", RuntimeWarning)


In [21]:
trimesh_seams = convert_to_trimesh(mesh_seams,add_texture_info=True)

/home/nikolas/Programs/miniconda3/envs/blender-tissue-cartography/lib/python3.11/site-packages/trimesh/grouping.py:99: RuntimeWarning: invalid value encountered in cast
  stacked = np.column_stack(stacked).round().astype(np.int64)


In [22]:
convert_from_trimesh(trimesh_seams, reconstruct_texture_from_faces=False).texture_vertices.shape

(8159, 2)

In [23]:
convert_from_trimesh(trimesh_seams, reconstruct_texture_from_faces=False).faces

[[[3, 3], [2, 2], [29, 29]],
 [[4, 4], [29, 29], [0, 0]],
 [[1, 1], [6, 6], [4, 4]],
 [[7, 7], [32, 32], [2, 2]],
 [[29, 29], [4, 4], [3, 3]],
 [[4, 4], [6, 6], [5, 5]],
 [[4, 4], [0, 0], [1, 1]],
 [[8, 8], [7, 7], [2, 2]],
 [[2, 2], [3, 3], [8, 8]],
 [[5, 5], [6, 6], [36, 36]],
 [[7, 7], [11, 11], [10, 10]],
 [[8, 8], [11, 11], [7, 7]],
 [[9, 9], [5, 5], [36, 36]],
 [[13, 13], [9, 9], [36, 36]],
 [[12, 12], [9, 9], [13, 13]],
 [[10, 10], [11, 11], [14, 14]],
 [[38, 38], [12, 12], [13, 13]],
 [[15, 15], [12, 12], [38, 38]],
 [[14, 14], [18, 18], [40, 40]],
 [[16, 16], [15, 15], [38, 38]],
 [[17, 17], [15, 15], [16, 16]],
 [[41, 41], [17, 17], [16, 16]],
 [[20, 20], [42, 42], [18, 18]],
 [[20, 20], [43, 43], [42, 42]],
 [[17, 17], [23, 23], [19, 19]],
 [[23, 23], [17, 17], [41, 41]],
 [[43, 43], [20, 20], [44, 44]],
 [[22, 22], [44, 44], [20, 20]],
 [[92, 92], [44, 44], [22, 22]],
 [[19, 19], [23, 23], [21, 21]],
 [[23, 23], [24, 24], [21, 21]],
 [[21, 21], [24, 24], [22, 22]],
 [[22, 2

In [24]:
mesh_seams_reconverted = convert_from_trimesh(trimesh_seams, reconstruct_texture_from_faces=True)

In [25]:
mesh_seams.texture_vertices.shape, mesh_seams_reconverted.texture_vertices.shape

((8288, 2), (8288, 2))

In [26]:
mesh_seams_reconverted.write_obj("drosophila_example/Drosophila_CAAX-mCherry_mesh_uv_resaved_trimesh.obj")